## Configuração e Dados

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# --- GERAÇÃO DE DADOS ---
# 200 Pacientes, 1000 Genes. Apenas os primeiros 20 são informativos.
X, y = make_classification(
    n_samples=200, n_features=1000, n_informative=20,
    n_redundant=0, n_repeated=0, n_classes=2,
    random_state=42, shuffle=False
)

gene_names = [f"Gene_{i}" for i in range(X.shape[1])]
df = pd.DataFrame(X, columns=gene_names)

# Split Treino/Teste (Fundamental para validar a seleção)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Escalar (Obrigatório para alguns métodos)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Dados gerados: {X_train.shape} (Treino).")
df.head()

### 2. Filter Method: Seleção Estatística Univariada (SelectKBest)

Os métodos de filtro (**Filter Methods**) são os mais rápidos e simples. Eles avaliam cada variável individualmente em relação à variável alvo (`y`), calculam uma nota (score) e selecionam apenas as melhores.

Neste exemplo, usamos o **SelectKBest**, que mantém as $k$ variáveis com os maiores scores estatísticos.

**Passo a passo do código:**
1.  **Definição da Métrica (`score_func`):** Usamos o `f_classif`, que realiza um teste ANOVA (Análise de Variância). Ele verifica se a média do gene é significativamente diferente entre os pacientes doentes e saudáveis.
2.  **Seleção (`k`):** Definimos `k=50` para manter apenas os 50 genes com maior pontuação.
3.  **Visualização:** O gráfico de barras mostra o "F-Score" de cada gene.

---

#### Como variar os parâmetros (Experimente!):

* **`k` (Número de Features):**
    * Tente alterar `k=50` para `k=10` ou `k=200`.
    * Se `k='all'`, ele retorna as notas de todas as variáveis sem filtrar nada (bom para análise exploratória).

* **`score_func` (Função de Pontuação):**
    * **`f_classif`:** Usado aqui (Ideal para: *Input Numérico -> Target Categórico*).
    * **`chi2`:** Use para dados não-negativos, como contagem de palavras (Ideal para: *Input Categórico -> Target Categórico*).
    * **`mutual_info_classif`:** Captura relações não-lineares (mais pesado computacionalmente).
    * **`f_regression`:** Se o seu problema fosse prever um número contínuo (Regressão), não uma classe.

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif

# Selecionar os top 50 genes baseados em estatística ANOVA (f_classif)
selector_filter = SelectKBest(score_func=f_classif, k=50) # Outras score functions
selector_filter.fit(X_train, y_train)

# Quais foram escolhidos? (Máscara booleana)
mask = selector_filter.get_support()
selected_genes_filter = np.array(gene_names)[mask]

print("--- FILTER METHOD ---")
print(f"Top 5 genes selecionados: {selected_genes_filter[:5]}")

# Visualizando os Scores dos primeiros 100 genes
scores = selector_filter.scores_
plt.figure(figsize=(12, 4))
plt.bar(range(100), scores[:100])
plt.title("Scores Estatísticos (ANOVA) - Primeiros 100 Genes")
plt.xlabel("Índice do Gene")
plt.ylabel("F-Score")
plt.show()
print("Nota: O filtro detectou bem o pico nos primeiros 20 genes!")

### 3. Wrapper Method: Eliminação Recursiva de Features (RFE)

Os métodos Wrapper ("Envelopados") utilizam um modelo preditivo real para avaliar a importância das variáveis. O **RFE (Recursive Feature Elimination)** funciona como um processo de "seleção natural":

1.  Treina o modelo com todas as variáveis.
2.  Identifica as variáveis menos importantes (menores coeficientes ou *feature importance*).
3.  Remove ("poda") essas variáveis do conjunto.
4.  Repete o processo até sobrar apenas o número desejado de variáveis.

**Passo a passo do código:**
1.  **O Estimador Base (`estimator`):** Definimos uma `LogisticRegression`. É este modelo que vai decidir quem fica e quem sai a cada rodada. Aumentamos `max_iter` para garantir que modelo irá convergir mesmo com muitos dados.
2.  **O Seletor RFE:** Configuramos o RFE para usar a Regressão Logística e parar quando restarem apenas **20 genes**.
3.  **O Passo (`step=0.1`):** Para acelerar o processo (já que temos 1000 colunas), configuramos para remover **10%** das piores features a cada iteração, em vez de remover apenas uma por vez.



---

#### Como variar os parâmetros (Experimente!):

* **`estimator` (O Juiz):**
    * Tente trocar `LogisticRegression` por `RandomForestClassifier()` ou `LinearSVC()`.
    * *Nota:* O modelo escolhido **precisa** ter os atributos `coef_` ou `feature_importances_` (modelos como KNN ou Naive Bayes não funcionam aqui).

* **`step` (Velocidade vs. Precisão):**
    * **`step=1`:** Remove 1 variável por vez. É o mais preciso, mas extremamente lento (treinará o modelo 980 vezes!).
    * **`step=0.1` a `0.5`:** Remove blocos de variáveis (10% a 50%). Muito mais rápido, ideal para Big Data.

* **`n_features_to_select` (A Meta):**
    * Define quantos "sobreviventes" você quer no final. Se você não souber o número ideal, existe uma variação chamada **RFECV** que usa validação cruzada para descobrir o número ótimo automaticamente.

In [ ]:
from sklearn.feature_selection import RFE, RFECV
from sklearn.linear_model import LogisticRegression

# Usaremos Regressão Logística como base
estimator = LogisticRegression(max_iter=1000, solver='liblinear')

# RFE: Quero que sobrem apenas 20 genes
# step=0.1 significa remover 10% das features a cada iteração (para ser rápido)
rfe = RFE(estimator=estimator, n_features_to_select=20, step=0.1)
#rfe_cv = RFECV(estimator=estimator, step=0.1)
rfe.fit(X_train_scaled, y_train)

selected_genes_rfe = np.array(gene_names)[rfe.support_]

print("--- WRAPPER METHOD (RFE) ---")
print(f"Genes selecionados pelo RFE: {selected_genes_rfe}")

### 4. Embedded Method: Seleção Baseada em Árvores (Random Forest)

Os métodos embarcados (**Embedded Methods**) realizam a seleção de atributos como parte integrante do processo de treinamento do modelo. Eles são mais eficientes que os Wrappers e mais precisos que os Filters.

O **Random Forest** é o exemplo clássico. Ele constrói centenas de árvores de decisão.
* Em cada nó de cada árvore, o algoritmo escolhe a variável que melhor separa as classes (reduz a impureza/Gini).
* Se uma variável é escolhida muitas vezes e no topo das árvores, ela ganha uma **Importância** alta.
* Se uma variável é ruído, ela quase nunca é escolhida, ficando com importância próxima de zero.



**Passo a passo do código:**
1.  **Treinamento:** Instanciamos e treinamos o `RandomForestClassifier`. Note que não usamos nenhuma função de seleção separada (como `RFE` ou `SelectKBest`). O próprio `.fit()` calcula tudo.
2.  **Extração (`feature_importances_`):** O modelo possui um atributo interno que guarda a soma da redução de impureza de cada variável.
3.  **Ordenação (`argsort`):** Como o array de importâncias vem na ordem original das colunas (Gene_0, Gene_1...), usamos o `argsort` do NumPy com `[::-1]` para ordenar do maior para o menor e descobrir os "campeões".

---

#### Como variar os parâmetros (Experimente!):

* **`n_estimators` (Número de Árvores):**
    * O padrão é 100.
    * Aumentar para `500` ou `1000` torna a estimativa da importância mais estável e confiável (reduz a variância da escolha), mas demora mais para treinar.

* **`criterion` (Critério de Divisão):**
    * **`gini`:** (Padrão) Mede a impureza de Gini. Geralmente mais rápido.
    * **`entropy`:** Mede o Ganho de Informação. Às vezes seleciona atributos ligeiramente diferentes. Vale testar se a lista de genes muda.

* **`max_depth` (Profundidade da Árvore):**
    * Se você limitar a profundidade (ex: `max_depth=3`), força o modelo a escolher apenas as variáveis **super cruciais** (as que aparecem no topo), ignorando interações complexas e profundas.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Treino
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Importâncias
importances = rf.feature_importances_
indices = np.argsort(importances)[::-1] # Ordenar decrescente

print("--- EMBEDDED METHOD (Random Forest) ---")
print("Top 5 Genes mais importantes:")
for i in range(5):
    print(f"{gene_names[indices[i]]}: {importances[indices[i]]:.4f}")

# Gráfico
plt.figure(figsize=(10, 4))
plt.title("Feature Importance (Random Forest) - Top 30")
plt.bar(range(30), importances[indices[:30]], align="center")
plt.xticks(range(30), [gene_names[i] for i in indices[:30]], rotation=90)
plt.show()

### 5. Extração de Atributos: PCA (Principal Component Analysis)

Diferente dos métodos anteriores que **descartam** variáveis (Seleção), o PCA **mantém todas elas**, mas de uma forma compactada. Ele realiza uma transformação matemática (álgebra linear) para criar "novos eixos" que resumem a informação.

Imagine que temos 1000 genes.
* **Seleção (RFE/Lasso):** Escolhe 2 genes e joga 998 no lixo.
* **Extração (PCA):** Cria 2 "Super-Variáveis" (PC1 e PC2). O PC1 é uma mistura ponderada de todos os 1000 genes.

**Passo a passo do código:**
1.  **Definição (`n_components=2`):** Pedimos ao PCA para reduzir o mundo de 1000 dimensões para apenas 2. Por que 2? Porque queremos visualizar em uma tela (eixo X e Y).
2.  **O Input (`X_train_scaled`):**
    *  **Atenção Crítica:** O PCA é extremamente sensível à escala dos dados. Se você não usar o `StandardScaler` antes, o PCA vai considerar que variáveis com números grandes (ex: salário) são mais importantes que variáveis com números pequenos (ex: idade). **Sempre padronize antes do PCA.**
3.  **A Transformação:** O método `.fit_transform()` calcula a matriz de rotação e já aplica a projeção, retornando as coordenadas dos pacientes nesse novo mundo 2D.



---

#### Como variar os parâmetros (Experimente!):

* **`n_components` (O Grau de Compressão):**
    * **Número Inteiro (ex: `2`, `3`, `50`):** Define exatamente quantas colunas você quer na saída. Use 2 ou 3 para gráficos.
    * **Número Decimal (ex: `0.95`, `0.99`):** Isso muda o comportamento! Em vez de pedir "X colunas", você diz: *"Mantenha colunas suficientes para preservar **95%** da informação original"*. O algoritmo decide sozinho se precisa de 10, 50 ou 200 componentes. É muito usado em Machine Learning real.

* **`svd_solver`:**
    * Para datasets gigantescos, você pode usar `svd_solver='randomized'`, que faz uma aproximação estatística muito mais rápida do que o cálculo exato.

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_train_scaled) # PCA sempre nos dados escalados!

plt.figure(figsize=(8, 6))
sns.scatterplot(x=X_pca[:,0], y=X_pca[:,1], hue=y_train, palette='viridis', style=y_train, s=100)
plt.title("PCA: Projeção 2D dos Pacientes")
plt.xlabel("PC1 (Maior Variância)")
plt.ylabel("PC2 (Segunda Maior)")
plt.show()

### Determinando o Número Ideal de Componentes

Antes de transformarmos os dados, precisamos responder à pergunta de ouro: **"Quantos componentes devemos manter?"**

Não precisamos chutar um número. Podemos calcular exatamente quantos eixos são necessários para preservar uma certa porcentagem da informação original (geralmente 90%, 95% ou 99%).

Isso é feito analisando a **Variância Explicada Acumulada**:
1.  O PC1 explica X% (ex: 40%).
2.  O PC1 + PC2 explicam Y% (ex: 40% + 20% = 60%).
3.  Continuamos somando até atingir nossa meta (ex: 95%).

No código abaixo, vamos gerar o **Scree Plot** (Gráfico de Cotovelo) para visualizar essa soma e descobrir o ponto de corte ideal.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# 1. Instanciar PCA sem limitar o número de componentes
# (Ou limitando ao máximo possível, que é o min(n_samples, n_features))
pca_full = PCA(n_components=None)
pca_full.fit(X_train_scaled)

# 2. Calcular a Variância Acumulada
cumulative_variance = np.cumsum(pca_full.explained_variance_ratio_)

# 3. Determinar matematicamente o corte (Ex: 95%)
limite_desejado = 0.95
n_components_ideal = np.argmax(cumulative_variance >= limite_desejado) + 1

print(f"Para preservar {limite_desejado*100}% da informação, precisamos de {n_components_ideal} componentes.")

# 4. Plotar o Scree Plot
plt.figure(figsize=(10, 5))
plt.plot(cumulative_variance, marker='o', linestyle='--', color='b')
plt.axhline(y=limite_desejado, color='r', linestyle='-', label=f'Corte de {limite_desejado*100}%')
plt.axvline(x=n_components_ideal-1, color='r', linestyle='--')
plt.xlabel('Número de Componentes')
plt.ylabel('Variância Explicada Acumulada')
plt.title('Scree Plot: Quantos componentes guardar?')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Em vez de escolher '2' ou '10', escolhemos '0.95' (95%)
pca_auto = PCA(n_components=0.95)
X_pca_auto = pca_auto.fit_transform(X_train_scaled)

print(f"O PCA selecionou automaticamente {pca_auto.n_components_} componentes para atingir 95% de variância.")
print(f"Shape original: {X_train_scaled.shape}")
print(f"Shape reduzido: {X_pca_auto.shape}")

### Visualizando a Escolha do Número de Componentes (Zoom nos Top 30)

Muitas vezes, visualizar todos os componentes (ex: 1000) torna o gráfico ilegível. Por isso, focamos apenas nos primeiros (Top 30), onde a maior parte da ação acontece.

O código abaixo gera dois gráficos fundamentais para a sua tomada de decisão:

1.  **Gráfico da Esquerda (Individual - O "Cotovelo"):**
    * Mostra quanto cada componente contribui isoladamente.
    * **O que procurar:** O ponto onde a curva deixa de cair bruscamente e passa a ficar "plana". Isso indica que os componentes seguintes são apenas ruído.

2.  **Gráfico da Direita (Acumulada - A "Meta"):**
    * Mostra a soma da informação retida.
    * **O que procurar:** O ponto exato onde a linha laranja cruza a linha vermelha pontilhada (nossa meta de 95%).
    * O código desenhará uma **linha verde** vertical indicando o número ideal ($k$) para essa meta.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Definindo um limite visual (ex: mostrar apenas os top 30 para não espremer o gráfico)
limit = 30
variance_ratio = pca_full.explained_variance_ratio_[:limit]
cumulative_variance = np.cumsum(pca_full.explained_variance_ratio_)[:limit]

# Configurando a figura com 2 gráficos lado a lado
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# --- GRÁFICO 1: O COTOVELO (Individual) ---
# Mostra onde a informação "despenca"
ax1.bar(range(1, len(variance_ratio)+1), variance_ratio, alpha=0.7, color='steelblue', label='Variância Individual')
ax1.plot(range(1, len(variance_ratio)+1), variance_ratio, 'r.-') # Linha para destacar a queda
ax1.set_title('Scree Plot: Onde está o Cotovelo?')
ax1.set_xlabel('Componentes Principais')
ax1.set_ylabel('Variância Explicada')
ax1.grid(True, linestyle='--', alpha=0.6)
ax1.legend()

# --- GRÁFICO 2: A SOMA (Acumulada) ---
# Mostra quando atingimos a meta (ex: 90%)
ax2.plot(range(1, len(cumulative_variance)+1), cumulative_variance, marker='o', linestyle='-', color='darkorange', linewidth=2)
ax2.set_title('Variância Acumulada: Quando parar?')
ax2.set_xlabel('Componentes Principais')
ax2.set_ylabel('Total de Informação (%)')

# Linha de Corte (Threshold) de 95%
threshold = 0.95
ax2.axhline(y=threshold, color='red', linestyle='--', label=f'Meta de {threshold*100}%')

# Encontrar o ponto exato onde cruzou 95%
k_ideal = np.argmax(cumulative_variance >= threshold) + 1
ax2.axvline(x=k_ideal, color='green', linestyle='--', label=f'Ideal: {k_ideal} Comps')

ax2.legend()
ax2.grid(True, linestyle='--', alpha=0.6)

plt.tight_layout()
plt.show()

print(f"Conclusão Visual: O 'cotovelo' acontece cedo, mas para ter {threshold*100}% de precisão precisamos de {k_ideal} componentes.")

### Cálculo Matemático do Corte (Sem "Olhômetro")

Embora o gráfico ajude a ter uma intuição, em ciência de dados precisamos de reprodutibilidade.

Se não conseguirmos identificar visualmente o cotovelo, ou se quisermos automatizar o processo, usamos a seguinte lógica:

1.  Definimos uma meta (Threshold), por exemplo, **0.95 (95%)**.
2.  Percorremos a lista de variância acumulada.
3.  Paramos no **primeiro componente** que fizer a soma ultrapassar 0.95.

O código abaixo faz isso instantaneamente usando a função `np.argmax`.

In [ ]:
import numpy as np

# 1. Definimos nossa meta de qualidade (ex: 95% da informação original)
target_variance = 0.95

# 2. Calculamos a variância acumulada (Soma progressiva)
cumulative_variance = np.cumsum(pca_full.explained_variance_ratio_)

# 3. O "Pulo do Gato": np.argmax retorna o PRIMEIRO índice que satisfaz a condição
# Somamos +1 porque índices em Python começam em 0 (o componente 1 é índice 0)
n_components_ideal = np.argmax(cumulative_variance >= target_variance) + 1

print(f"--- CÁLCULO AUTOMÁTICO ---")
print(f"Para explicar {target_variance*100}% da variância, precisamos de: {n_components_ideal} componentes.")
print(f"Variância real atingida com {n_components_ideal} componentes: {cumulative_variance[n_components_ideal-1]*100:.2f}%")

### Definindo a Meta: Otimização Automática

Em vez de "chutar" 90% ou 95%, podemos perguntar ao computador:
*"Qual é o número de componentes que faz o meu modelo de Câncer ter a maior acurácia possível?"*

Isso transforma o número de componentes em um **Hiperparâmetro** a ser otimizado.

O código abaixo usa o `GridSearchCV` para testar várias quantidades de componentes e nos dizer qual funcionou melhor para a classificação final.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# 1. Criar um "Tubo" (Pipeline) que conecta o PCA ao Modelo
pca = PCA()
logistic = LogisticRegression(max_iter=1000)
pipe = Pipeline(steps=[('pca', pca), ('logistic', logistic)])

# 2. Definir os testes: Vamos testar 2, 5, 10, 15, 20 componentes... até 30
param_grid = {
    'pca__n_components': [2, 5, 10, 15, 20, 25, 30]
}

# 3. Rodar a busca (Grid Search) com Validação Cruzada (cv=5)
search = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy')
search.fit(X_train_scaled, y_train)

print("Melhor número de componentes calculado:", search.best_params_)
print("Acurácia atingida:", search.best_score_)